In [1]:
import os
from clearml import Task

from dotenv import load_dotenv

# Package import
from sample_project import module

PROJECT_ROOT = os.path.abspath(os.path.join(os.curdir, os.pardir))

print(f"Project root: {PROJECT_ROOT}")

Project root: /home/aleksey/Dev/MLOpsSandbox/clearml


In [2]:
load_dotenv();

# You can use next string for ClearML credentials for local execution but there are some issues with remote execution
# os.environ["CLEARML_CONFIG_FILE"] = os.path.join(PROJECT_ROOT, "clearml.conf")

In [3]:
# Modified example from https://github.com/catalyst-team/catalyst#getting-started

import os
from torch import nn, optim
from torch.utils.data import DataLoader
from catalyst import dl, utils
from catalyst.contrib.datasets import MNIST


def start_training(clearml_task: Task):
    model = nn.Sequential(nn.Flatten(), nn.Linear(28 * 28, 10))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.02)
    loaders = {
        "train": DataLoader(MNIST(os.getcwd(), train=True), batch_size=32),
        "valid": DataLoader(MNIST(os.getcwd(), train=False), batch_size=32),
    }

    runner = dl.SupervisedRunner(
        input_key="features", output_key="logits", target_key="targets", loss_key="loss"
    )

    num_epochs = 100

    if clearml_task.running_locally():
        print('Running locally')
        num_epochs = 0
    else:
        print('Running remotely')

    # model training
    runner.train(
        model=model,
        criterion=criterion,
        optimizer=optimizer,
        loaders=loaders,
        num_epochs=num_epochs,
        callbacks=[
            dl.AccuracyCallback(input_key="logits", target_key="targets", topk=(1, 3, 5)),
            dl.PrecisionRecallF1SupportCallback(input_key="logits", target_key="targets"),
        ],
        logdir="./logs",
        valid_loader="valid",
        valid_metric="loss",
        minimize_valid_metric=True,
        verbose=True,
    )

/home/aleksey/Dev/MLOpsSandbox/clearml/sample_project/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
clearml_task = Task.init(project_name="sample_project", reuse_last_task_id=False)

try:
    clearml_task.add_dependency(".")
    start_training(clearml_task=clearml_task)
    clearml_task.execute_remotely(queue_name="my-queue")
    # Should be after remote execution under `try`
    clearml_task.mark_completed()
except Exception as e:
    print(f"Failed to execute: {e}")
    clearml_task.mark_failed()

ClearML Task: created new task id=fcf6b450428f4310995184a5b0c18127
ClearML results page: http://localhost:8080/projects/9c71843452994a3c983b37ba594627fa/experiments/fcf6b450428f4310995184a5b0c18127/output/log
2023-01-30 01:35:36,465 - clearml.Task - INFO - Storing jupyter notebook directly as code
2023-01-30 01:35:36,544 - clearml.model - INFO - Selected model id: 567fc43ed8774139b42ca2bfe6d3721b
2023-01-30 01:35:36,876 - clearml.model - INFO - Selected model id: d9282f83fef74be29e703ea036d5071a


/home/aleksey/Dev/MLOpsSandbox/clearml/sample_project/.venv/lib/python3.8/site-packages/catalyst/contrib/datasets/mnist.py:125: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = torch.tensor(self.data)


Running locally
Top models:

ClearML results page: http://localhost:8080/projects/9c71843452994a3c983b37ba594627fa/experiments/fcf6b450428f4310995184a5b0c18127/output/log
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring


Switching to remote execution, output log page http://localhost:8080/projects/9c71843452994a3c983b37ba594627fa/experiments/fcf6b450428f4310995184a5b0c18127/output/log


: 

: 